In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
np.set_printoptions(precision=3)

import pandas as pd

from TCGAMultiOmics.multiomics import MultiOmicsData

from moge.network.heterogeneous_network import HeterogeneousNetwork
from moge.network.omics_distance import compute_expression_correlations, compute_annotation_similarity
# from moge.visualization. import matrix_heatmap

# Heterogeneous Network

In [2]:
# READ Edgeslist
# network.import_edgelist_file(file="moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_node_similarity_99_50.edgelist", 
#                              directed=False)
# network.import_edgelist_file(file="moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_regulatory_interactions.edgelist", 
#                              directed=True)
# network.remove_extra_nodes()

In [23]:
# WRITE Edgelist
nx.write_edgelist(network_train.get_subgraph(["MIR", "GE", "LNC"]), 
                  "moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_mi-lnc_filtered.edgelist", 
                  data=False)

In [13]:
import pickle



# READ
with open('moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_mi-lnc_filtered.pickle', 'rb') as file:
    network = pickle.load(file)


#  Split Network to train test and write to file

In [15]:
from moge.evaluation.utils import split_train_test_network

network_train, val_edges_dict, test_edges_dict = split_train_test_network(network, network.node_list, 
                                                          test_frac=.01, val_frac=.15, edge_types=["u", "d", "u_n"], 
                                                          seed=0)

Removed 3276 test, and 49142 val, type u edges
Removed 4891 test, and 73373 val, type d edges
Removed 4861 test, and 72921 val, type u_n edges


# Create a new network with test edges

In [5]:
folder_path = "/home/jonny/PycharmProjects/Bioinformatics_ExternalData/tcga-assembler/LUAD/"
external_data_path = "/home/jonny/PycharmProjects/Bioinformatics_ExternalData/"
luad_data = MultiOmicsData(cancer_type="LUAD", tcga_data_path=folder_path, external_data_path=external_data_path,
                           modalities=[\
                                       "GE",
                                       "MIR",
                                       "LNC",
                                       # "CNV",
                                       # "SNP",
#                                        "PRO",
                                       # "DNA"
                                       ])

/home/jonny/.conda/envs/jonny_conda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


PATIENTS (522, 110)
DRUGS (461, 4)
GE (576, 20284)
MIR (494, 1870)
LNC (546, 12727)
SAMPLES (599, 111)
Processed genes info for  GE
Processed genes info for  MIR


/home/jonny/.conda/envs/jonny_conda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:615: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")
/home/jonny/.conda/envs/jonny_conda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:309: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.HGNC_lncrna_info.groupby("symbol").first(), on="symbol",
/home/jonny/.conda/envs/jonny_conda3/lib/python3.6/site-packages/pandas/core/frame.py:6336: FutureWarning: 'Gene Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  rsuffix=rsuffix, sort=sort)


Processed genes info for  LNC


In [17]:
from moge.network.heterogeneous_network import HeterogeneousNetwork

network_test = HeterogeneousNetwork(modalities=["MIR", "GE", "LNC"], multi_omics_data=luad_data)
network_val = HeterogeneousNetwork(modalities=["MIR", "GE", "LNC"], multi_omics_data=luad_data)
node_list = network_train.node_list

MIR  nodes: 1870
GE  nodes: 20284
LNC  nodes: 12725
Total nodes: 34879
MIR  nodes: 1870
GE  nodes: 20284
LNC  nodes: 12725
Total nodes: 34879


In [18]:
for edge_type, edges in test_edges_dict.items():
    for edge in edges:
        network_test.G.add_edge(node_list[edge[0]], node_list[edge[1]], type=edge_type)
print(network_test.G.number_of_edges())

13028


In [19]:
for edge_type, edges in val_edges_dict.items():
    for edge in edges:
        network_val.G.add_edge(node_list[edge[0]], node_list[edge[1]], type=edge_type)
print(network_val.G.number_of_edges())

195436


# Write train and test networks to file

In [20]:
# WRITE
with open('moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_filtered_train.pickle', 'wb') as file:
    pickle.dump(network_train, file)
    
with open('moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_filtered_test.pickle', 'wb') as file:
    pickle.dump(network_test, file)
    
with open('moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_filtered_val.pickle', 'wb') as file:
    pickle.dump(network_val, file)